<a href="https://colab.research.google.com/github/dropcreations/7zip-in-Google-Colab/blob/main/7zip-in-Google-Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# __Mount Google Drive__

In [ ]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3><b>Mount Google Drive</b></h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

# __7-Zip__

* Run below cell to **Install 7-Zip** to the runtime

In [ ]:
#@markdown <br><center><img src='https://raw.githubusercontent.com/dropcreations/7zip-in-Google-Colab/main/7zip-Logo.png' height="50" alt="7zip-logo"/></center>
#@markdown <center><h3><b>Install 7-Zip</b></h3></center><br>
from IPython.display import clear_output
!sudo apt update
!sudo apt install p7zip-full p7zip-rar unrar rar
clear_output()
print("Successfully Installed.")

### __Compress Files and Folders__

* Create a **zip file** from a **file**
* Add **Source file path** and **Output file path**.
* Don't need to add file **extention** in output file path, you can select it in **"File_Type"**.

In [ ]:
Source_File = "" #@param {type:"string"}
Output_File = "" #@param {type:"string"}
File_Type = "zip" #@param ["zip", "7z", "tar", "gzip", "bzip2", "xz", "wim"]

!7z a -t{File_Type} "{Output_File}".zip "{Source_File}"

* Create a **zip, tar, 7z** file from a **folder**.
* If you want you can add **password** or **split** the archive.
* If you want to save archive in **another** location **uncheck** "Save_to_the_source_folder_location".

In [ ]:
import os, sys, re

Source_Folder = "" #@param {type:"string"}
File_format = "zip" #@param ["zip", "7z", "tar"]
Password = "" #@param {type:"string"}
Split = "no" #@param ["no", "10m", "100m", "500m", "1g", "2g"] {allow-input: true}
Compress_level = "9" #@param ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
Save_to_the_source_folder_location = True #@param {type:"boolean"}

output_file_path = re.search("^[\/].+\/", Source_Folder)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", Source_Folder)
folder_name = re.sub("^[\/]", "", delsplit.group(0))

os.environ['inputDir'] = Source_Folder
os.environ['outputPath'] = output_file_path_raw
os.environ['folderName'] = folder_name
os.environ['fileType'] = File_format
os.environ['passADD'] = Password
os.environ['splitVOL'] = Split
os.environ['level'] = Compress_level

if Save_to_the_source_folder_location == True:
    if Split == "no":
      if Password == "":
        !7z a -t$fileType -mx="$level" "$outputPath"/"$folderName" "$inputDir"
      else:
        !7z a -t$fileType -p"$passADD" -mx="$level" "$outputPath"/"$folderName" "$inputDir"
    else:
      if Password == "":
        !7z a -t$fileType -mx="$level" -v"$splitVOL" "$outputPath"/"$folderName" "$inputDir"
      else:
        !7z a -t$fileType -p"$passADD" -mx="$level" -v"$splitVOL" "$outputPath"/"$folderName" "$inputDir"
else:
    Output_folder = input("Enter output folder path: ")
    os.environ['outFolder'] = Output_folder
    if Split == "no":
      if Password == "":
        !7z a -t$fileType -mx="$level" "$outFolder"/"$folderName" "$inputDir"
      else:
        !7z a -t$fileType -p"$passADD" -mx="$level" "$outFolder"/"$folderName" "$inputDir"
    else:
      if Password == "":
        !7z a -t$fileType -mx="$level" -v"$splitVOL" "$outFolder"/"$folderName" "$inputDir"
      else:
        !7z a -t$fileType -p"$passADD" -mx="$level" -v"$splitVOL" "$outFolder"/"$folderName" "$inputDir"

### __Uncompress Files__

* Run below cell to **list content** of archive.

In [ ]:
File_Path = "" #@param {type:"string"}
!7z l "{File_Path}"

* Can also extract **splited** archives.
* Type **archive file path** and **extract folder path** to extract file.
* If you want to extract files from archive **without using directory names**, check below.
* NOTE : **Don't check** below at normal use.

In [ ]:
File_Path = "" #@param {type:"string"}
Extract_Folder = "" #@param {type:"string"}
extract_without_directory_names = False #@param {type:"boolean"}

import os, sys, re

os.environ['inputFile'] = File_Path
os.environ['outputFile'] = Extract_Folder

if extract_without_directory_names == False:
    !7z e "$inputFile" -spf -o"$outputFile"
else:
    !7z e "$inputFile" -o"$outputFile"